In [2]:
import numpy as np
#import hrr_cobot_robot
import spatialmath as sm
import rospy
np.set_printoptions(precision=5, suppress=True)

In [12]:
np.deg2rad(70)

1.2217304763960306

In [2]:
#This is usually unnecessary
def get_hostname_and_IP():
    import socket
    with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as s:
        s.connect(("8.8.8.8", 80))
        return socket.gethostname(), s.getsockname()[0]

_host, _ip = get_hostname_and_IP()
%env ROS_MASTER_URI=http://127.0.0.1:11311
# %env ROS_MASTER_URI=http://hrrcobotLinux54:11311
print(f"\t\033[1m\033[4m!!!please check that the IPs below is identical!!!\033[0m"+
      f"\n\thost {_host} has IP {_ip}, which should be identical to",end="\nvs:\t")
%env ROS_IP=$_ip

env: ROS_MASTER_URI=http://127.0.0.1:11311
	!!!please check that the IPs below is identical!!!
	host hrrN3511rt2004 has IP 192.168.2.34, which should be identical to
vs:	env: ROS_IP=192.168.2.34


In [13]:
rospy.init_node("NotebookDev_Changepositions2")

In [3]:
cobot = hrr_cobot_robot.HrrCobotControl.from_ros("/hrr_cobot")

[INFO][/NotebookDev_Changepositions2] Initialize MATLAB-Motion planner @ /home/schrottgott/_ros/hr_recycler_ws/src/planner/src
[INFO][/NotebookDev_Changepositions2] connected to Matlab
Shutting down global node /matlab_global_node_08706 with NodeURI http://127.0.0.1:33003/
The value of the ROS_IP environment variable, 127.0.0.1, will be used to set the advertised address for the ROS node.
> In ros.internal/Net/generateNodeHost (line 368)
In ros/Node/parseArguments (line 349)
In rosinit (line 89)
In HrRecyclerDataImport (line 82)
Initializing global node /matlab_global_node_20836 with NodeURI http://127.0.0.1:34217/
[INFO][/NotebookDev_Changepositions2] MATLAB-Initialization done


pybullet build time: May  8 2021 05:48:13


[INFO]some functions are not yet compiled. Expect delays upon first call


In [4]:
print(cobot)

In [42]:
cobot.T_B_E_robot, np.rad2deg(cobot.q)

(   0.7603   -0.6495   -0.005813 -0.3938    
  -0.6495   -0.7603   -0.0067   -0.4635    
  -6.751e-05  0.00887  -1         0.6155    
   0         0         0         1         
,
 array([130.35019,  27.05725, -70.172  ,   0.00533,  82.26169, -90.15816]))

In [19]:
cobot.stop()

# Brief introduction

The tool change routine consists of two parts: pickup and disposal of a tool. For each tool, a sequence of positions for both pickup and dispose is recorded manually and saved in a python dictionary, with some additional information. Whenever a tool is changed with the actual action service, the corresponding dictionary is loaded and the robot moves through the positions in sequential order. The dictionaries look like this:
```
vacuum_2 = {
    "pickup": 
        {"poses_A": pickA, "poses_q": pickq,
        "close_tc": pickc, "use_joint_ctrl": pickj},
     "dispose":
        {"poses_A": dispA, "poses_q": dispq,
        "open_tc": dispo, "use_joint_ctrl": dispj}
    }
np.save("vacuum_2", vacuum_2)
```
Here the name vacuum_2 indicates that the dict concerns the vacuum gripper placed at the 2nd toolrack position.

vacuum2["pickup"] is a dict that contains several lists of equal length:

vacuum2["pickup"]["poses_A"]: A list of spatialmath SE3 objects with the pose of the robot

vacuum2["pickup"]["poses_q"]: A list of np.arrays with the corresponding joints of the robot

vacuum2["pickup"]["close_tc"]: A list of 0s and a single 1 indicating wether tool changer should close after arrival

vacuum2["pickup"]["use_joint_ctrl"]: A list of 0s and 1s indicating wether the robot should move linearly in joint space instead of in cartesian, in order to reach the position.

Here is an example of the pickup routine in action:
```
x = vacuum_2["pickup"]
for i in range(len(x["close_tc"])):
    if x["use_joint_ctrl"][i]:
        cobot.move_to_joint_pose(x["poses_q"][i])
    else:
        cobot.goTo(x["poses_A"][i],v_max=0.01)
    if x["close_tc"][i]:   
        #close tool changer

```
dispose routine is similiar, only "open_tc" instead of "close_tc".

At the beginning, the robot is in calibration pose with base joint turned so that the robot faces the begining of the toolrack.


# Record a fully new routine

## Record the pickup routine


### [Optional] Place robot in starting position - The first saved position should be reachable from here or anywhere above the tool rack without collision.

In [66]:
cobot.stop()

In [69]:
cobot.q_calib

array([ 1.22173,  0.     , -1.5708 ,  0.     ,  1.5708 ,  0.     ])

In [9]:
start=cobot.q_calib
start[0]=np.deg2rad(70)

In [68]:
cobot.move_to_joint_pose(start)
#cobot.goTo(cobot.FK(start),v_max = 0.05)

KeyboardInterrupt: 

### Initialize lists, move robot manually and append info to the lists at each important pose.

In [70]:
pick_A = []
pick_q = []
pick_close_tc = []
pick_joint_ctrl = []

In [71]:
# Run this cell for each position you want saved in the sequence
cobot.update_tf()
pick_A.append(cobot.T_B_E_robot)
pick_q.append(cobot.q)
pick_close_tc.append(0) #Set to 1 if the current pose is the one where tool changer is closed
pick_joint_ctrl.append(0) #Set to 1 if the current pose was reached by joint control
pick_A

[   1        -0.0003671  8.641e-05  0.001808  
  -0.0003663 -1        -0.009783 -0.6099    
   8.999e-05  0.009783 -1         0.6255    
   0         0         0         1         
]

Make sure that the last position is safe such that the robot can go to calibration pose without collision!

## Disposal

In [ ]:
start=cobot.q_calib
start[0]=np.deg2rad(70)
#cobot.move_to_joint_pose(start)
cobot.goTo(cobot.FK(start),v_max = 0.05)

In [10]:
disp_A = []
disp_q = []
disp_open_tc = []
disp_joint_ctrl = []

In [12]:
cobot.update_tf()
disp_A.append(cobot.T_B_E_robot)
disp_q.append(cobot.q)
disp_open_tc.append(0) #Set to 1 if the current pose is the one where tool changer is opened
disp_joint_ctrl.append(0) #Set to 1 if the current pose was reached by joint control
disp_A

[   0.9998    0.01894  -0.0009315 -0.1726    
   0.01894  -0.9998   -0.00102  -0.3516    
  -0.0009506  0.001002 -1         0.6064    
   0         0         0         1         
,
   -0.4017    0.9158    0.001377  0.3909    
   0.9158    0.4017   -0.0004991 -0.02424   
  -0.00101   0.001061 -1         0.6064    
   0         0         0         1         
]

In [76]:
disp_A = [disp_A[0],disp_A[2],disp_A[1],disp_A[2],disp_A[3]]
disp_q = [disp_q[0],disp_q[2],disp_q[1],disp_q[2],disp_q[3]]
disp_open_tc = [0,0,1,0,0]
disp_joint_ctrl = [disp_joint_ctrl[0],disp_joint_ctrl[2],disp_joint_ctrl[1],disp_joint_ctrl[2],disp_joint_ctrl[3]]

In [29]:
disp_joint_ctrl

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]

In [49]:
disp_joint_ctrl.append(disp_joint_ctrl[1])

In [50]:
disp_joint_ctrl.append(temp)

In [85]:
disp_A

[   0.5724   -0.82     -0.006029 -0.4622    
  -0.82     -0.5723   -0.00605  -0.5227    
   0.00151   0.008407 -1         0.6122    
   0         0         0         1         
,
    0.5724   -0.82     -0.00603  -0.4622    
  -0.82     -0.5723   -0.00605  -0.5227    
   0.00151   0.008407 -1         0.3878    
   0         0         0         1         
,
    0.5724   -0.82     -0.006029 -0.4622    
  -0.82     -0.5723   -0.00605  -0.5227    
   0.00151   0.008407 -1         0.344     
   0         0         0         1         
,
    0.5724   -0.82     -0.00603  -0.4622    
  -0.82     -0.5723   -0.00605  -0.5227    
   0.00151   0.008407 -1         0.3878    
   0         0         0         1         
,
    0.5724   -0.82     -0.006024 -0.4622    
  -0.82     -0.5723   -0.006045 -0.5227    
   0.001509  0.0084   -1         0.5209    
   0         0         0         1         
]

In [19]:
disp_joint_ctrl[-1]=1

In [52]:
disp_open_tc[1] = 1

In [53]:
disp_open_tc

[0, 1, 0, 0]

In [11]:
np.save("nextCutting_q", disp_q, allow_pickle=True)

Make sure that the last position is safe such that the robot can go to calibration pose or
anywhere above the toolracks without collision


## Save the routine
Possibly change the name in the last line from "vacuum_2" to shaftgrinder_1, wsg_3, ... see directory or below. Be careful with overwriting!

In [30]:
x = {
    "pickup": 
        {"poses_A": list(reversed(disp_A)), "poses_q": list(reversed(disp_q)),
        "close_tc": list(reversed(disp_open_tc)), "use_joint_ctrl": list(reversed(disp_joint_ctrl))},
     "dispose":
        {"poses_A": disp_A, "poses_q": disp_q,
        "open_tc": disp_open_tc, "use_joint_ctrl": disp_joint_ctrl}
    }
np.save("screw_6", x, allow_pickle=True)

In [60]:
len(x["dispose"]["poses_q"])

15

In [34]:
np.save("vacuum_2_hardcode", x, allow_pickle=True)

In [ ]:
grasping contact y = -0.607

In [74]:
x2 = x

In [77]:
x2["pickup"]["poses_q"]

[array([ 2.9051466 ,  0.29345739, -1.81037557,  0.60714132, -0.18333299,
        -3.45537257]),
 array([ 2.90504074,  0.28983906, -1.80648589,  0.64240432, -0.65010577,
        -5.26182222]),
 array([ 2.90504146,  0.96812749, -1.87894857,  0.38404006, -1.31662977,
        -4.82586575]),
 array([ 2.88268065,  0.99662572, -1.79855442,  0.36640158, -1.26576483,
        -4.83883572]),
 array([ 2.87448835,  1.01280844, -1.75472021,  0.36108482, -1.23889971,
        -4.84634781]),
 array([ 2.8744874 ,  1.00742269, -1.75699496,  0.36146629, -1.23599207,
        -4.84750748]),
 array([ 2.87443399,  1.00896859, -1.75632143,  0.35988504, -1.24490702,
        -4.85847902]),
 array([ 2.8751781 ,  1.01032758, -1.75133693,  0.36109811, -1.24161899,
        -4.86009359]),
 array([ 2.87517834,  0.97678345, -1.76483834,  0.36359876, -1.2228837 ,
        -4.86762142]),
 array([ 2.87855506,  0.96840447, -1.78639972,  0.36547631, -1.23566985,
        -4.86390972]),
 array([ 2.9157002 ,  0.91445351, -1.931

In [67]:
x["pickup"]["poses_q"][-3] = np.deg2rad(x["pickup"]["poses_q"][-3])
x["pickup"]["poses_q"][-2] = np.deg2rad(x["pickup"]["poses_q"][-2])
x["pickup"]["poses_q"][-1] = np.deg2rad(x["pickup"]["poses_q"][-1])

In [68]:
x["dispose"]["poses_q"][0] = np.deg2rad(x["dispose"]["poses_q"][0])
x["dispose"]["poses_q"][1] = np.deg2rad(x["dispose"]["poses_q"][1])
x["dispose"]["poses_q"][2] = np.deg2rad(x["dispose"]["poses_q"][2])

In [83]:
x["pickup"]["poses_A"][2].t[1]=-0.607

In [88]:
x["dispose"]["poses_A"]

[   0.9998    0.01809   0        -0.1731    
   0.01809  -0.9998    0        -0.6088    
   0         0        -1         0.6069    
   0         0         0         1         
,
    0.9998    0.01808   0        -0.1731    
   0.01808  -0.9998    0        -0.6088    
   0         0        -1         0.375     
   0         0         0         1         
,
    0.9998    0.01808   0        -0.1731    
   0.01808  -0.9998    0        -0.607     
   0         0        -1         0.3448    
   0         0         0         1         
,
    0.9998    0.01808   0        -0.1731    
   0.01808  -0.9998    0        -0.6088    
   0         0        -1         0.375     
   0         0         0         1         
,
    0.9998    0.01808   0        -0.1731    
   0.01808  -0.9998    0        -0.6088    
   0         0        -1         0.5672    
   0         0         0         1         
]

# Load a routine to manipulate/test it

In [2]:
import numpy as np
import spatialmath as sm

In [14]:
x = np.load("shaftgrinder_1.npy",allow_pickle=True).item()
#x = np.load("vacuum_2.npy",allow_pickle=True).item()
#x = np.load("wsg_3.npy",allow_pickle=True).item()
#x = np.load("pgrip_4.npy",allow_pickle=True).item()
#x = np.load("screw_6.npy",allow_pickle=True).item()

In [3]:
x = np.load("vacuum_2_hardcode.npy",allow_pickle=True).item()

In [ ]:
x["pickup"]

In [18]:
np.rad2deg(x["pickup"]["poses_q"])

KeyError: 'Poses_A'

In [6]:
np.rad2deg(x["pickup"]["poses_q"][0])

array([ 8.98301562e+01,  2.70874750e+01, -7.00557340e+01, -3.73387482e-03,
        8.22915671e+01, -9.01902984e+01])

In [23]:
x3["pickup"]["poses_A"] = x["pickup"]["poses_A"][0:-1]
x3

[SE3:  -0.602815    0.633079    0.485618    0.177893     
        0.257509    0.730433   -0.632579   -0.349415     
       -0.755184   -0.256277   -0.60334     0.592501     
        0           0           0           1            
     ,
 SE3:  -0.999923    0.0110554   0.00563426  0.178912     
       -0.00582283 -0.0170996  -0.999837   -0.463505     
       -0.0109573  -0.999793    0.0171627   0.433721     
        0           0           0           1            
     ,
 SE3:  -0.999922    0.0111457   0.00564105  0.178907     
       -0.00583097 -0.0170816  -0.999837   -0.4635       
       -0.0110475  -0.999792    0.0171453   0.229111     
        0           0           0           1            
     ,
 SE3:  -0.999922    0.0111518   0.00563372  0.178904     
       -0.00582384 -0.0170892  -0.999837   -0.516334     
       -0.0110538  -0.999792    0.0171528   0.229114     
        0           0           0           1            
     ,
 SE3:  -0.999922    0.0111513   0.00563372  

In [6]:
np.rad2deg(np.array([-0.08106, -0.23017, -2.33784, -0.93423, -0.8107, 2.32261]))

array([  -4.6444 ,  -13.18777, -133.94837,  -53.52744,  -46.44969,
        133.07575])

In [4]:
x["pickup"]["poses_q"]

[array([ 1.13527, -0.2772 , -2.18416, -0.50619,  0.36575,  0.93848]),
 array([ 1.13527,  0.29975, -2.226  , -0.50612, -1.03444,  1.84371]),
 array([ 1.13528,  0.86198, -2.16947, -0.43139, -1.48784,  1.60573]),
 array([ 1.18258,  0.90272, -2.02446, -0.38926, -1.3892 ,  1.64059]),
 array([ 1.18258,  0.85257, -2.04206, -0.39173, -1.3591 ,  1.65322]),
 array([ 1.16511,  0.83311, -2.10115, -0.40645, -1.39714,  1.64123]),
 array([ 1.15472,  0.81551, -2.15735, -0.41486, -1.43397,  1.62735]),
 array([ 1.04685,  0.7067 , -2.38477, -0.51879, -1.54506,  1.58445]),
 array([ 0.97625,  0.68623, -2.50067, -0.53262, -1.62764,  1.5366 ]),
 array([ 0.97625,  0.00962, -2.51749, -0.61716, -1.06752,  1.89982]),
 array([ 0.91198, -0.33526, -2.41279, -0.93165, -0.77443,  2.3376 ]),
 array([ 0.70181,  0.01387, -2.12119, -1.08044, -0.95781,  2.40092]),
 array([ 0.88324, -0.03279, -2.16738,  0.57401, -0.66543,  1.08265]),
 array([-0.08106, -0.23017, -2.33784, -0.93423, -0.8107 ,  2.32261])]

In [10]:
x["pickup"]["use_joint_ctrl"]

[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [12]:
beforedispose_afterpickup = np.r_[158, -18, -125, 2, -54, -180]

In [13]:
totheleft = np.r_[37, -18, -125, 2, -54, -180]

In [14]:
after_before_unscrewing = np.r_[21.8, -8, -105, 0, 65, -173]

In [15]:
mylist_q = [beforedispose_afterpickup, totheleft, after_before_unscrewing]

In [14]:
disp_A, disp_q, disp_joint_ctrl

([   0.9998    0.01894  -0.0009315 -0.1726    
   0.01894  -0.9998   -0.00102  -0.3516    
  -0.0009506  0.001002 -1         0.6064    
   0         0         0         1         
,
    -0.4017    0.9158    0.001377  0.3909    
   0.9158    0.4017   -0.0004991 -0.02424   
  -0.00101   0.001061 -1         0.6064    
   0         0         0         1         
],
 [array([ 2.02708, -0.12164, -1.93933,  0.0004 ,  1.32257, -1.09567]),
  array([ 0.06186, -0.12166, -1.93935,  0.00041,  1.32257, -1.09568])],
 [0, 0])

In [31]:
x["dispose"]["use_joint_ctrl"][0]

1

In [27]:
qq = cobot.q
AA = cobot.T_B_E_robot
qq,AA

(array([ 0.04133, -0.00231, -1.57311, -0.     ,  1.57079, -1.57646]),
    0.04698   0.9989    0         0.4347    
   0.9989   -0.04698   0        -0.01798   
   0         0        -1         0.705     
   0         0         0         1         
)

In [32]:
x["pickup"]["poses_q"][-1]

array([ 1.55094, -0.00231, -1.57311, -0.     ,  1.5708 , -1.57647])

In [33]:
x["dispose"]["poses_A"].insert(0,AA)
x["dispose"]["poses_q"].insert(0,qq)
x["dispose"]["use_joint_ctrl"].insert(0,0)
x["dispose"]["open_tc"].insert(0,0)
x["pickup"]["poses_A"].append(AA)
x["pickup"]["poses_q"].append(qq)
x["pickup"]["use_joint_ctrl"].append(1)
x["pickup"]["close_tc"].append(0)

In [15]:
x["dispose"]["poses_A"].insert(0,disp_A[1])
x["dispose"]["poses_A"].insert(0,disp_A[0])
x["dispose"]["poses_q"].insert(0,disp_q[1])
x["dispose"]["poses_q"].insert(0,disp_q[0])
x["dispose"]["use_joint_ctrl"].insert(0,1)
x["dispose"]["use_joint_ctrl"].insert(0,0)
x["dispose"]["open_tc"].insert(0,0)
x["dispose"]["open_tc"].insert(0,0)

In [16]:
x["pickup"]["poses_A"].append(disp_A[0])
x["pickup"]["poses_A"].append(disp_A[1])
x["pickup"]["poses_q"].append(disp_q[0])
x["pickup"]["poses_q"].append(disp_q[1])
x["pickup"]["use_joint_ctrl"].append(0)
x["pickup"]["use_joint_ctrl"].append(1)
x["pickup"]["close_tc"].append(0)
x["pickup"]["close_tc"].append(0)

In [17]:
x["pickup"]

{'poses_A': [   0.9998    0.01808   0        -0.1731    
   0.01808  -0.9998    0        -0.6088    
   0         0        -1         0.5672    
   0         0         0         1         
,
     0.9998    0.01808   0        -0.1731    
   0.01808  -0.9998    0        -0.6088    
   0         0        -1         0.375     
   0         0         0         1         
,
     0.9998    0.01808   0        -0.1731    
   0.01808  -0.9998    0        -0.607     
   0         0        -1         0.3448    
   0         0         0         1         
,
     0.9998    0.01808   0        -0.1731    
   0.01808  -0.9998    0        -0.6088    
   0         0        -1         0.375     
   0         0         0         1         
,
     0.9998    0.01809   0        -0.1731    
   0.01809  -0.9998    0        -0.6088    
   0         0        -1         0.6069    
   0         0         0         1         
,
     0.9998    0.01894  -0.0009315 -0.1726    
   0.01894  -0.9998   -0.00102  -0.3516   

In [24]:
x["pickup"]["poses_q"].append(beforedispose_afterpickup)
x["pickup"]["poses_q"].append(totheleft)
x["pickup"]["poses_q"].append(after_before_unscrewing)

In [40]:
x["dispose"]["use_joint_ctrl"].insert(0,1)

In [43]:
x["dispose"]["open_tc"].insert(0,0)

In [50]:
x["pickup"]["poses_A"].append(sm.SE3())

In [ ]:
#A small helper function to shift a pose's position by vec = [x y z] in m
def moveByVec(vec, pose):
    #shift position of se3 by vec
    return sm.SE3(vec)@pose

### Before automatic testing, check if automatic open/close tool changer works!

In [14]:
cobot.close_tool_changer()#force=True)

In [13]:
cobot.open_tool_changer()#force=True)

In [12]:
cobot.stop()

In [1]:
cobot.T_B_E_robot

NameError: name 'cobot' is not defined

### Pickup
Make sure that tool changer is open!

In [ ]:
#open tool changer

In [10]:
cobot.move_to_joint_pose(start)
# cobot.goTo(cobot.FK(start),v_max = 0.05)

In [24]:
poses = x["dispose"]
#print(poses["close_tc"])

In [29]:
x["dispose"]["poses_q"][0]

array([ 1.55094, -0.00231, -1.57311, -0.     ,  1.5708 , -1.57647])

In [25]:
#Manual loop
i = 0 #change to 1,2,3...
if poses["use_joint_ctrl"][i]:
    cobot.move_to_joint_pose(poses["poses_q"][i]) 
else:
    cobot.goTo(poses["poses_A"][i],v_max=0.01) 
if poses["close_tc"][i]:
    rospy.sleep(0.5)
    cobot.close_tool_changer(force=True)

KeyError: 'close_tc'

In [56]:
#Automatic runthrough
for i in range(len(poses["poses_A"])):
    if poses["use_joint_ctrl"][i]:
        rospy.sleep(0.5)
        cobot.move_to_joint_pose(poses["poses_q"][i]) 
    else:
        rospy.sleep(0.5)
        cobot.goTo(poses["poses_A"][i],v_max=0.01) 
    if poses["close_tc"][i]:
        rospy.sleep(0.5)
        cobot.close_tool_changer(force=True)

### Dispose

In [57]:
poses = x["dispose"]
print(poses)

In [20]:
x["dispose"]["poses_A"]

[   0.9999    0.01419   0         0.00864   
   0.01419  -0.9999    0        -0.435     
   0         0        -1         0.705     
   0         0         0         1         
,
    0.9999    0.01419   0         0.005041  
   0.01419  -0.9999    0        -0.4986    
   0         0        -1         0.3532    
   0         0         0         1         
,
    0.9999    0.01419   0         0.005042  
   0.01419  -0.9999    0        -0.5342    
   0         0        -1         0.3509    
   0         0         0         1         
,
    0.9999    0.01422   0         0.001555  
   0.01422  -0.9999    0        -0.5561    
   0         0        -1         0.35      
   0         0         0         1         
,
    0.9999    0.01422   0         0.001557  
   0.01422  -0.9999    0        -0.6087    
   0         0        -1         0.35      
   0         0         0         1         
,
    0.9999    0.01422   0         0.001557  
   0.01422  -0.9999    0        -0.6087    
   0         0  

In [17]:
#Manual loop
i = 0 #change to 1,2,3...
if poses["use_joint_ctrl"][i]:
    cobot.move_to_joint_pose(poses["poses_q"][i]) 
else:
    cobot.goTo(poses["poses_A"][i],v_max=0.01) 
if poses["open_tc"][i]:
    rospy.sleep(0.5)
    cobot.open_tool_changer(force=True)

KeyboardInterrupt: 

In [58]:
#Automatic runthrough
for i in range(len(poses["poses_A"])):
    if poses["use_joint_ctrl"][i]:
        rospy.sleep(0.5)
        cobot.move_to_joint_pose(poses["poses_q"][i]) 
    else:
        rospy.sleep(0.5)
        cobot.goTo(poses["poses_A"][i],v_max=0.01) 
    if poses["open_tc"][i]:
        rospy.sleep(0.5)
        cobot.open_tool_changer(force=True)

In [7]:
twoCutting_q = np.load("twoCutting_q.npy")

In [14]:
nextCutting_q = np.load("nextCutting_q.npy")

In [8]:
twoCutting_q[0]

array([ 0.59162,  0.24501, -1.86247, -1.63102,  1.49799,  3.68177])

In [10]:
cobot.move_to_joint_pose(twoCutting_q[0]) 

In [14]:
cobot.move_to_joint_pose(twoCutting_q[1]) 

In [ ]:
cobot.move_to_joint_pose(twoCutting_q[1]) 